In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [257]:
# R  W  111111
# R  P  11111
# G  W  11111
# G  P  11111
# H  W  11111
# H  P  1111

# category = 'R'
category = 'G'
# category = 'H'

# bet_type = 'W'
bet_type = 'P'

In [258]:
engine = create_engine('sqlite:///tab.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

14024 races loaded!


In [259]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

5689 G races!


In [260]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head()

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,results,runners
0,1,RICHMOND,NSW,C,G,2017-09-13,1,LADBROKES INFO HUB MDN HT1,2017-09-13 05:12:00,Paying,400,"[[4], [2], [1], [5]]",8,"[{""runnerName"": ""ZALI'S BLUEBOY"", ""runnerNumbe...","[[4], [2], [1], [5]]","[{'runnerName': 'ZALI'S BLUEBOY', 'runnerNumbe..."
1,2,RICHMOND,NSW,C,G,2017-09-13,2,KEVIN WATERS TOWING MDN HT2,2017-09-13 05:32:00,Paying,400,"[[1], [8], [7], [2]]",8,"[{""runnerName"": ""SKETCHY LINDA"", ""runnerNumber...","[[1], [8], [7], [2]]","[{'runnerName': 'SKETCHY LINDA', 'runnerNumber..."
2,3,RICHMOND,NSW,C,G,2017-09-13,4,PENRITHELECTRICAL.COM.AU STKS,2017-09-13 06:07:00,Paying,400,"[[9], [7], [8], [2]]",8,"[{""runnerName"": ""BROWZA BOB'"", ""runnerNumber"":...","[[9], [7], [8], [2]]","[{'runnerName': 'BROWZA BOB'', 'runnerNumber':..."
3,4,RICHMOND,NSW,C,G,2017-09-13,5,RICHMONDGREYHOUNDS.COM.AU STKS,2017-09-13 06:27:00,Paying,618,"[[6], [7], [3], [1]]",8,"[{""runnerName"": ""MIAMI DEMON"", ""runnerNumber"":...","[[6], [7], [3], [1]]","[{'runnerName': 'MIAMI DEMON', 'runnerNumber':..."
4,5,RICHMOND,NSW,C,G,2017-09-13,6,SWIFT FANCY @ STUD 1-2W STKS,2017-09-13 06:42:00,Paying,535,"[[6], [7], [1], [8]]",8,"[{""runnerName"": ""KILLING TIME"", ""runnerNumber""...","[[6], [7], [1], [8]]","[{'runnerName': 'KILLING TIME', 'runnerNumber'..."


In [261]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
#     print(race)

    num_runners = len([r for r in race['runners'] if r['win_odds'] and r['place_odds']])

    # add num_runners to all runners
    for runner in race['runners']:
#         print(runner)
        runner['num_runners'] = num_runners
        runner['race_type'] = race['race_type']

    data_all = data_all.append(race['runners'])
    
data_all.tail(10)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,num_runners,parimutuel,place_odds,race_type,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
8,0.440117,0.146300,0.153435,0.161800,1,0,3,"{'returnWin': 6, 'returnWinOpen': 6.5, 'return...",8,"{'bettingStatus': 'Normal', 'returnWin': 5.4, ...",1.85,G,,TOO DARDY,1,BARRY MCPHERSON,6.0,0.166667,4,0.139167
0,0.246223,0.077715,0.051263,0.050722,5,0,4,"{'returnWin': 16, 'returnWinOpen': 10, 'return...",8,"{'bettingStatus': 'Normal', 'returnWin': 11.7,...",3.45,G,,RED HOT REAGAN,5,ENZO CRUDELI,16.0,0.062500,7,0.051507
1,0.550778,0.173840,0.198446,0.196349,6,0,0,"{'returnWin': 4.2, 'returnWinOpen': 3.2, 'retu...",8,"{'bettingStatus': 'Normal', 'returnWin': 5.4, ...",1.60,G,,IMMORTALITY,6,BEN MACLEAN,4.2,0.238095,2,0.196216
2,0.408801,0.129028,0.118999,0.117741,7,0,0,"{'returnWin': 7.5, 'returnWinOpen': 8.5, 'retu...",8,"{'bettingStatus': 'Normal', 'returnWin': 8.4, ...",2.15,G,,HOWZAT ERNIE,7,JEN THOMPSON,7.5,0.133333,4,0.109881
3,0.370326,0.116885,0.104817,0.103709,8,0,2,"{'returnWin': 8.5, 'returnWinOpen': 9.5, 'retu...",8,"{'bettingStatus': 'Normal', 'returnWin': 7.8, ...",2.30,G,,ISSUE,8,MICHAEL JOHNSTON,8.5,0.117647,5,0.096954
4,0.000000,0.000000,0.000000,0.000000,0,0,0,"{'returnWin': 0, 'returnWinOpen': 0, 'returnPl...",8,"{'bettingStatus': 'Scratched', 'returnWin': 0,...",0.00,G,,FLYING ZARLI,9,BEN MACLEAN,0.0,0.000000,9,0.000000
5,0.657444,0.207507,0.299634,0.296467,1,0,1,"{'returnWin': 2.6, 'returnWinOpen': 3.6, 'retu...",8,"{'bettingStatus': 'Normal', 'returnWin': 2.7, ...",1.33,G,,REALITY BOMBER,1,TERRY ERENSHAW,2.6,0.384615,1,0.316964
6,0.307873,0.097173,0.081215,0.080357,2,0,0,"{'returnWin': 11, 'returnWinOpen': 7, 'returnP...",8,"{'bettingStatus': 'Normal', 'returnWin': 9.2, ...",2.70,G,,ROTTEN RENDER,2,JOHN IWANYK,11.0,0.090909,6,0.074919
7,0.184007,0.058077,0.033428,0.033075,3,0,3,"{'returnWin': 23, 'returnWinOpen': 15, 'return...",8,"{'bettingStatus': 'Normal', 'returnWin': 23.3,...",4.45,G,,MISS WARNER,3,GLEN PRICE,23.0,0.043478,8,0.035831
8,0.442848,0.139775,0.122880,0.121581,4,0,0,"{'returnWin': 7, 'returnWinOpen': 5, 'returnPl...",8,"{'bettingStatus': 'Normal', 'returnWin': 7.3, ...",2.10,G,,TANGO DANCER,4,BARRY MCPHERSON,7.0,0.142857,3,0.117729


In [262]:
# drop scratched
data = data_all.dropna(subset=['win_odds', 'place_odds'])
data = data[(data.win_odds > 0)]
data.describe()

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,num_runners,place_odds,runnerNumber,win_odds,win_perc,win_rank,win_scaled
count,4.344900e+04,4.344900e+04,4.344900e+04,4.344900e+04,43449.000000,43449.0,43449.000000,43449.000000,43449.000000,43449.000000,43449.000000,43449.000000,43449.000000,43449.000000
mean,3.662377e-01,1.309351e-01,1.309644e-01,1.309351e-01,4.393266,0.0,1.251628,7.707381,3.074830,4.610210,13.202415,0.162413,4.281410,0.130935
std,1.873710e-01,6.983266e-02,1.171897e-01,1.155688e-01,2.273911,0.0,1.459103,0.729309,21.645561,2.434308,13.573202,0.138425,2.228202,0.110125
min,2.731574e-13,1.482145e-13,1.789280e-31,1.510060e-31,0.000000,0.0,0.000000,2.000000,0.000000,1.000000,1.010000,0.000000,1.000000,0.000000
25%,2.165582e-01,7.622961e-02,4.290262e-02,4.290878e-02,2.000000,0.0,0.000000,8.000000,1.680000,3.000000,4.400000,0.055556,2.000000,0.046250
50%,3.466903e-01,1.279567e-01,1.031353e-01,1.025548e-01,4.000000,0.0,1.000000,8.000000,2.300000,5.000000,8.000000,0.125000,4.000000,0.098456
75%,4.977278e-01,1.727516e-01,1.772133e-01,1.777250e-01,6.000000,0.0,2.000000,8.000000,3.450000,7.000000,18.000000,0.227273,6.000000,0.181006
max,9.886354e-01,7.410312e-01,9.999999e-01,9.086142e-01,8.000000,0.0,4.000000,10.000000,2250.000000,10.000000,151.000000,0.990099,10.000000,0.774723


In [263]:
# get label data
Y = data['finishingPosition']

if bet_type == 'W':
    Y = (Y == 1)
elif bet_type == 'P':
    Y = (Y == 1) | (Y == 2) | ((Y == 3) & (data['num_runners'] >= 8))

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

count    43449.000000
mean         0.361458
std          0.480428
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
dtype: float64

In [264]:
# xp = fixed odds perc
xp = data['win_perc']
xp.describe()

count    43449.000000
mean         0.162413
std          0.138425
min          0.000000
25%          0.055556
50%          0.125000
75%          0.227273
max          0.990099
Name: win_perc, dtype: float64

In [265]:
# xs = odds scaled
xs = data['win_scaled']
xs.describe()

count    43449.000000
mean         0.130935
std          0.110125
min          0.000000
25%          0.046250
50%          0.098456
75%          0.181006
max          0.774723
Name: win_scaled, dtype: float64

In [266]:
# xr = win rank
xr = data['win_rank']
xr.describe()

count    43449.000000
mean         4.281410
std          2.228202
min          1.000000
25%          2.000000
50%          4.000000
75%          6.000000
max         10.000000
Name: win_rank, dtype: float64

In [267]:
# xn = get num runners
xn = data['num_runners']
xn.describe()

count    43449.000000
mean         7.707381
std          0.729309
min          2.000000
25%          8.000000
50%          8.000000
75%          8.000000
max         10.000000
Name: num_runners, dtype: float64

In [268]:
X = pd.concat([xp, xs, xr, xn], axis=1)
X.describe()
# sns.pairplot(pd.concat([X, Y], axis=1))

,win_perc,win_scaled,win_rank,num_runners
count,43449.000000,43449.000000,43449.000000,43449.000000
mean,0.162413,0.130935,4.281410,7.707381
std,0.138425,0.110125,2.228202,0.729309
min,0.000000,0.000000,1.000000,2.000000
25%,0.055556,0.046250,2.000000,8.000000
50%,0.125000,0.098456,4.000000,8.000000
75%,0.227273,0.181006,6.000000,8.000000
max,0.990099,0.774723,10.000000,10.000000


In [269]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 1000
print('epochs = {}'.format(epochs))

tag = '{}30x30{}'.format(category, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
except OSError:
    model = Sequential()
    model.add(Dense(30, activation='relu', input_dim=n))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

input dimension = 4
epochs = 1000
tag = G30x30P


In [270]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)